In [48]:
import json
import MySQLdb
import string

db = MySQLdb.connect("localhost","root","","podcast" )
cursor = db.cursor()
data = json.load(open('data/podcast_meta.json'))


for k in data:
    features='NULL'
    if "features" in data[k]:
        features=data[k]["features"]
    fea=str.replace(features[0].encode('utf-8'),'\'','')
    sql="SELECT * FROM podcast.topics WHERE topic_name= '%s' " % (fea)
    topic_id=0
    try:
        cursor.execute(sql)
        results = cursor.fetchall()
        if len(results) ==0:
            sql="INSERT INTO podcast.topics (topic_name) VALUES ('%s')" % (fea)
            cursor.execute(sql)
            db.commit()
    except:
        print 'llllllllllllllllllllll'
        print fea
    


for k in data:
    #print k #podcast_name
    if "author" in data[k]:
        author=data[k]["author"] #author
    else:
        author='NULL'
    author=str.replace(author.encode('utf-8'),'\'','')
    if "image" in data[k]:
        image=data[k]["image"] #image
    else:
        image='NULL'
        
    features='NULL'
    if "features" in data[k]:
        features=data[k]["features"]
    fea=str.replace(features[0].encode('utf-8'),'\'','')
    #url=data[k]["url"]
    name=str.replace(k.encode('utf-8'),'\'','')
    sql="SELECT * FROM podcast.topics WHERE topic_name= '%s' " % (fea)
    topic_id=0
    try:
        cursor.execute(sql)
        results = cursor.fetchall()
        for row in results:
            topic_id= row[0]
        sql="INSERT INTO podcast.podcasts (pd_name, description,topic_id, pd_link, img_link) \
                VALUES ('%s', '%s', '%d', '%s', '%s' )" % (name,author, topic_id,image,image)
        #print sql
        cursor.execute(sql)
        db.commit()
    except:
        print 'aaaaaaaaaaaaaaaaaaaaaaa'
        print name
        print author
        print topic_id
        print image
        # Rollback in case there is any error
        db.rollback()
    

aaaaaaaaaaaaaaaaaaaaaaa
Touré Show
Cadence13
9
https://is4-ssl.mzstatic.com/image/thumb/Music128/v4/0d/43/e4/0d43e4dd-e4d7-b0d0-169a-b375e1146245/source/170x170bb.jpg
aaaaaaaaaaaaaaaaaaaaaaa
The Players’ Tribune: R2C2
The Players Tribune
2
https://is4-ssl.mzstatic.com/image/thumb/Music118/v4/f6/79/86/f67986ca-f58b-23c0-c469-a8d81bc2ea22/source/170x170bb.jpg
aaaaaaaaaaaaaaaaaaaaaaa
Love Your Work – Creative Productivity | Solopreneur | Self Publishing
David Kadavy – best-selling author & creative entrepreneur interviews David Allen
45
https://is3-ssl.mzstatic.com/image/thumb/Music62/v4/5b/09/58/5b0958b7-a1d7-42aa-c177-7d7e677fcbdf/source/170x170bb.jpg
aaaaaaaaaaaaaaaaaaaaaaa
Oprah’s SuperSoul Conversations
Oprah
3
https://is5-ssl.mzstatic.com/image/thumb/Music128/v4/86/5b/66/865b6609-6eed-453b-486d-67a3c0cb5f5b/source/170x170bb.jpg
aaaaaaaaaaaaaaaaaaaaaaa
36 Questions – The Podcast Musical
Two-Up
23
https://is4-ssl.mzstatic.com/image/thumb/Music118/v4/5c/5c/2f/5c5c2f1c-433f-1712-2de3-1c

In [60]:
data1 = json.load(open('data/u_to_p.json'))
for k in data1:
    if len(data1[k]) ==1:
        continue
    mail=str(k)+'abc@gmail.com'
    pas='abc123'
    sql="INSERT INTO podcast.users (tweet_id, mail_id,password) \
                VALUES ('%s', '%s', '%s')" % (k,mail, pas)
    try:
        cursor.execute(sql)
        db.commit()
        
    except:
        print 'aaaaaaaaaaaaaaaaaaaaaaa'
        print name
    
    #print k #user
    #for p in data1[k]:
    #    print p #podcast tweeted by this user

In [61]:
for k in data1:
    if len(data1[k]) ==1:
        continue
    sql="SELECT * FROM podcast.users WHERE tweet_id= '%s'"% (k)
    try:
        cursor.execute(sql)
        results = cursor.fetchall()
        user_id=0
        for row in results:
            user_id= row[0]
        #print user_id
        for p in data1[k]:
            #print p
            name=str.replace(p.encode('utf-8'),'\'','')
            sql="SELECT * FROM podcast.podcasts WHERE pd_name= '%s'"% (name)
            cursor.execute(sql)
            result1 = cursor.fetchall()
            pd_id=0
            for row in result1:
                pd_id= row[0]
            if pd_id != 0:
                sql="INSERT INTO podcast.user_pd_mp (user_id,pd_id,likes) VALUES ('%d','%d',1)" % (user_id, pd_id)
                cursor.execute(sql)
                db.commit()
    except:
        print 'aaaaaaaaaaa'